Amphibian and Mammal Habitats Under Threat, Merged	bio.012

In [1]:
import boto3

import rasterio
import numpy as np

import os
import sys
import threading

Establish local and s3 file locations

In [2]:
# Local storage
local_tmp_folder = "C:/Users/Max81007/Desktop/Python/Resource_Watch/Raster/bio_012/"

tmp1 = local_tmp_folder + "bio_012_1_amphib.tif"
tmp2 = local_tmp_folder + "bio_012_2_mammal.tif"
merge_files = [tmp1, tmp2]

tmp_merge = local_tmp_folder + "bio_012_merge.tif"

# S3 storage
s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/bio_012_amphibian_mammalian_species_under_threat_merge/"

s3_file1 = "resourcewatch/raster/bio_012_1_amphibian_species_under_threat/bio_012_1_amphibian_species_under_threat_edit.tif"
s3_file2 = "resourcewatch/raster/bio_012_2_mammalian_species_under_threat/bio_012_2_mammalian_species_under_threat_edit.tif"

# Make sure these match the order of the merge_files above
s3_files_to_merge = [s3_file1, s3_file2]
band_ids = ["amphibian_species_under_threat","mammalian_species_under_threat" ]
s3_key_merge = s3_folder + "bio_012_amphibian_mammalian_species_under_threat_merge.tif"

# S3 services
s3_download = boto3.resource("s3")
s3_upload = boto3.client("s3")

# Helper function to view upload progress
class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write(
                    "\r%s  %s / %s  (%.2f%%)" % (
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

In [7]:
for ix, s3_file in enumerate(s3_files_to_merge):
    s3_download.meta.client.download_file(s3_bucket, s3_file, merge_files[ix])

<b>Regardless of any needed edits, upload original file</b>

<i>Upload tif to S3 folder</i>

http://boto3.readthedocs.io/en/latest/guide/s3-example-creating-buckets.html

<i>Monitor Progress of Upload</i>

http://boto3.readthedocs.io/en/latest/_modules/boto3/s3/transfer.html
https://boto3.readthedocs.io/en/latest/guide/s3.html#using-the-transfer-manager

Create a merged tif with both amphibian and mammal threat data, upload to s3

In [5]:
with rasterio.open(merge_files[0]) as src:
    kwargs = src.profile
    
kwargs.update(
    count=len(merge_files)
)
    
with rasterio.open(tmp_merge, 'w', **kwargs) as dst:
    for idx, file in enumerate(merge_files):
        print(idx)
        with rasterio.open(file) as src:

            band = idx+1
            windows = src.block_windows()

            for win_id, window in windows:
                src_data = src.read(1, window=window)
                dst.write_band(band, src_data, window=window)

0


KeyboardInterrupt: 

In [31]:
with rasterio.open(tmp_merge) as src:
    num_bands = src.profile["count"] + 1
    data = {}
    for i in range(1, num_bands):
        data[i] = src.read(i)

In [32]:
np.unique(data[1], return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35], dtype=int32),
 array([923226633,   6778229,   1875054,    753439,    264679,    136269,
            39772,     24785,      9204,      4638,      2271,      1897,
             1294,       768,       268,       155,        28,        55,
                1,         8,         6,        25,        71,        31,
               61,        87,        73,        60,        30,        61,
                4,        34,        10]))

In [33]:
np.unique(data[2], return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33], dtype=int32),
 array([169243367, 154184574,  53840830, 200040897, 263286993,  57000855,
         18392252,   6902183,   2093180,   2953281,   1290368,    607972,
           383123,    381171,    352998,    355560,    289393,    224771,
           184635,    223244,    242506,    188104,    111229,     88250,
            76795,     52540,     45673,     36178,     21961,     14899,
             7136,      1993,      1054,        35]))

Upload edited file to s3

In [34]:
s3.upload_file(tmp_merge, s3_bucket, s3_key_merge,
           Callback=ProgressPercentage(tmp_merge))

/Users/nathansuberi/Desktop/WRI_Programming/RW_Data/temp/bio_012_merge.tif  107597805 / 107597805.0  (100.00%)

In [35]:
s3_key_merge

'resourcewatch/bio_012_amphibian_mammalian_species_under_threat_merge/bio_012_amphibian_mammalian_species_under_threat_merge.tif'

In [1]:
os.environ["s3_key"] = "s3://wri-public-data/" + s3_key_merge
os.environ["gs_key"] = "gs://resource-watch-public/" + s3_key_merge
!gsutil cp $s3_key $gs_key

os.environ["asset_id"] = "users/resourcewatch/ene_015_solar_energy_potential"
!earthengine upload image --asset_id=$asset_id $gs_key

NameError: name 's3_key_merge' is not defined